In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 1

In [2]:
import scipy as sp
import scipy.io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy.random import choice
from decision_tree.decision_tree import DecisionTree
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
%aimport decision_tree

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [3]:
%aimport

Modules to reload:
decision_tree

Modules to skip:



In [4]:
def preprocess(mtx):
    pass

# Spam

In [5]:
spam = scipy.io.loadmat("spam-dataset/spam_data.mat")

In [6]:
spam_raw_train_X = spam['training_data']
spam_raw_train_y = spam['training_labels'].reshape((5172,))
spam_raw_test_X = spam['test_data']

In [7]:
from sklearn.cross_validation import train_test_split

In [8]:
?train_test_split

In [9]:
a1, b1, at, bt = train_test_split(spam_raw_train_X, spam_raw_train_y, test_size=.1)

In [10]:
print(len(a1), len(b1))

4654 518


In [30]:
%autoreload
dt = DecisionTree({
        "max_depth":20,
        "min_points": 10
    })
dt.train(a1, at)

scored = dt.score(b1, bt)
print(sum(scored)/len(scored))

0.793436293436


# Census

In [8]:
census = pd.read_csv("census-dataset/data.csv", na_values=["?"])

In [9]:
categoricals = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "sex", "native-country"]